# Pandas - Resampling a pokročilé agregace

## Co je resampling a proč je důležitý?

Resampling je proces změny frekvence časových dat. V meteorologii často potřebujeme:

- **Agregovat detailní data** - z hodinových měření vytvořit denní souhrny
- **Vypočítat statistiky za období** - měsíční úhrny srážek, průměrné teploty
- **Změnit časové rozlišení** - z 10minutových dat na hodinové
- **Analyzovat trendy** - roční průměry pro klimatologické studie

### Typy resamplingu:

1. **Downsampling** - snížení frekvence (např. hodiny → dny)
2. **Upsampling** - zvýšení frekvence (např. dny → hodiny)

## Přehled metod a funkcí

### Resampling metody

| Metoda | Popis | Příklad použití |
|--------|--------|-----------------|
| `.resample()` | Hlavní metoda pro resampling | `df.resample('D')` |
| `.asfreq()` | Změna frekvence bez agregace | `df.asfreq('H')` |

### Agregační funkce pro resample

| Funkce | Popis | Typické použití |
|--------|--------|-----------------|
| `.mean()` | Průměr | Průměrná denní teplota |
| `.sum()` | Součet | Denní úhrn srážek |
| `.max()` | Maximum | Maximální denní teplota |
| `.min()` | Minimum | Minimální denní teplota |
| `.first()` | První hodnota | Ranní měření |
| `.last()` | Poslední hodnota | Večerní měření |
| `.count()` | Počet hodnot | Počet měření |
| `.std()` | Směrodatná odchylka | Variabilita teploty |
| `.agg()` | Více funkcí najednou | Kombinace statistik |

### Parametry resample()

| Parametr | Popis | Příklad |
|----------|-------|---------|
| `rule` | Cílová frekvence | `'D'`, `'W'`, `'M'` |
| `closed` | Která strana intervalu | `'left'`, `'right'` |
| `label` | Označení intervalu | `'left'`, `'right'` |
| `offset` | Posun začátku | `pd.Timedelta(hours=6)` |

### Rolling (klouzavé) operace

| Metoda | Popis | Příklad použití |
|--------|--------|-----------------|
| `.rolling()` | Klouzavé okno | `df.rolling(window=7)` |
| `.expanding()` | Expandující okno | `df.expanding()` |

### Parametry rolling()

| Parametr | Popis | Příklad |
|----------|-------|---------|
| `window` | Velikost okna | `7`, `'7D'` |
| `center` | Centrovat okno | `True`, `False` |
| `min_periods` | Min. počet hodnot | `1` |

### Groupby s časovými daty

| Metoda/Objekt | Popis | Příklad použití |
|---------------|--------|-----------------|
| `pd.Grouper()` | Seskupení podle času | `pd.Grouper(freq='M')` |
| `.groupby()` | Seskupení dat | `df.groupby(pd.Grouper(freq='M'))` |

### Business day offsety

| Offset | Popis | Příklad použití |
|--------|--------|-----------------|
| `BDay` | Pracovní den | `pd.offsets.BDay(1)` |
| `BMonthEnd` | Konec pracovního měsíce | `pd.offsets.BMonthEnd(1)` |
| `BYearEnd` | Konec pracovního roku | `pd.offsets.BYearEnd(1)` |

## Downsampling - agregace do delších období

### Základní použití resample()

In [1]:
import pandas as pd                   # Pro práci s tabulkovými daty (DataFrame)
import numpy as np                    # Pro matematické funkce (pouze pro generování vzorových dat)

# ----- VYTVOŘENÍ HODINOVÝCH METEOROLOGICKÝCH DAT -----
# Namísto složitých výpočtů použijeme konkrétní naměřené hodnoty

# Vytvoření týdne hodinových meteorologických dat (7 dní × 24 hodin = 168 záznamů)
hodinova_data = pd.DataFrame({
    'cas': pd.date_range('2023-06-01', periods=24*7, freq='H'),
    # pd.date_range() vytvoří posloupnost časů od 1.6.2023 po 7 dní
    # freq='H' znamená hodinovou frekvenci (každou hodinu)
    # periods=24*7 = 168 záznamů (7 dní × 24 hodin)
    
    # Teploty - vzorová data pro celý týden (°C)
    'teplota': [
        # První den (1.6.2023) - typický letní den
        15.2, 14.8, 14.5, 14.1, 14.0, 14.3, 15.8, 17.5, 19.8, 22.1, 24.3, 26.2,
        27.8, 28.5, 28.9, 28.1, 27.0, 25.2, 23.1, 21.4, 19.8, 18.3, 17.1, 16.2,
        # Druhý den (2.6.2023) - teplejší den
        15.8, 15.4, 15.1, 14.9, 14.7, 15.0, 16.5, 18.3, 20.8, 23.4, 25.9, 27.8,
        29.2, 30.1, 30.5, 29.8, 28.7, 26.9, 24.8, 22.6, 20.9, 19.4, 18.2, 17.3,
        # Třetí den (3.6.2023) - chladnější, deštivý
        16.5, 16.1, 15.8, 15.6, 15.4, 15.7, 16.2, 17.1, 18.5, 19.8, 20.9, 21.7,
        22.3, 22.8, 22.5, 22.1, 21.6, 20.8, 19.9, 19.1, 18.4, 17.8, 17.3, 16.9,
        # Čtvrtý den (4.6.2023) - stabilní počasí
        16.4, 16.0, 15.7, 15.5, 15.3, 15.6, 17.0, 18.8, 21.2, 23.7, 25.8, 27.4,
        28.6, 29.2, 28.8, 28.0, 26.8, 25.0, 23.2, 21.6, 20.1, 18.9, 17.8, 17.0,
        # Pátý den (5.6.2023) - velmi teplý den
        16.8, 16.4, 16.1, 15.9, 15.7, 16.1, 17.8, 20.1, 22.9, 25.8, 28.2, 30.1,
        31.5, 32.2, 32.0, 31.2, 29.8, 27.9, 25.7, 23.8, 22.0, 20.6, 19.4, 18.5,
        # Šestý den (6.6.2023) - mírné ochlazení
        17.9, 17.5, 17.2, 17.0, 16.8, 17.2, 18.5, 20.3, 22.6, 24.9, 26.8, 28.3,
        29.4, 30.0, 29.7, 29.0, 27.8, 26.1, 24.3, 22.7, 21.2, 20.0, 19.0, 18.3,
        # Sedmý den (7.6.2023) - návrat k normálu
        17.8, 17.4, 17.1, 16.9, 16.7, 17.0, 18.3, 20.0, 22.2, 24.5, 26.4, 27.9,
        29.0, 29.6, 29.3, 28.6, 27.5, 25.8, 24.0, 22.4, 20.9, 19.7, 18.7, 18.0
    ],
    
    # Relativní vlhkost vzduchu v procentech
    'vlhkost': [
        # První den - normální vlhkost
        82, 85, 87, 89, 91, 88, 75, 68, 58, 52, 47, 43,
        41, 39, 38, 42, 48, 55, 63, 70, 76, 79, 81, 83,
        # Druhý den - nižší vlhkost (sušší vzduch)
        80, 83, 85, 87, 89, 86, 73, 65, 55, 48, 42, 38,
        35, 33, 32, 36, 42, 50, 58, 66, 72, 76, 78, 80,
        # Třetí den - vysoká vlhkost (deštivý den)
        85, 88, 90, 92, 94, 91, 88, 85, 82, 78, 75, 73,
        72, 70, 72, 75, 78, 82, 85, 87, 89, 90, 91, 92,
        # Čtvrtý den - střední vlhkost
        84, 87, 89, 91, 93, 90, 77, 69, 59, 53, 48, 44,
        42, 40, 41, 45, 51, 58, 66, 73, 78, 81, 83, 85,
        # Pátý den - velmi nízká vlhkost (horký den)
        79, 82, 84, 86, 88, 85, 72, 63, 52, 45, 39, 34,
        30, 28, 29, 33, 39, 47, 55, 63, 69, 74, 77, 79,
        # Šestý den - mírně vyšší vlhkost
        81, 84, 86, 88, 90, 87, 74, 66, 56, 49, 44, 40,
        38, 36, 37, 41, 47, 54, 62, 69, 75, 79, 81, 83,
        # Sedmý den - návrat k normálu
        83, 86, 88, 90, 92, 89, 76, 68, 58, 51, 46, 42,
        40, 38, 39, 43, 49, 56, 64, 71, 77, 80, 82, 84
    ],
    
    # Rychlost větru v km/h
    'rychlost_vetru': [
        # První den - mírný vítr
        3.2, 2.8, 2.5, 2.1, 1.9, 2.3, 4.1, 5.8, 7.2, 8.5, 9.1, 8.7,
        8.3, 7.9, 8.2, 8.8, 8.4, 7.6, 6.8, 5.9, 5.1, 4.3, 3.8, 3.5,
        # Druhý den - silnější vítr
        4.1, 3.7, 3.4, 3.0, 2.8, 3.2, 5.0, 6.9, 8.5, 9.8, 10.4, 10.1,
        9.7, 9.3, 9.6, 10.2, 9.8, 9.0, 8.2, 7.1, 6.3, 5.4, 4.8, 4.4,
        # Třetí den - bouřkový vítr
        5.8, 5.4, 5.1, 4.7, 4.5, 4.9, 6.8, 8.7, 10.3, 11.6, 12.2, 11.9,
        11.5, 11.1, 11.4, 12.0, 11.6, 10.8, 10.0, 8.9, 8.1, 7.2, 6.6, 6.2,
        # Čtvrtý den - klidnější
        3.8, 3.4, 3.1, 2.7, 2.5, 2.9, 4.7, 6.4, 7.8, 9.1, 9.7, 9.4,
        9.0, 8.6, 8.9, 9.5, 9.1, 8.3, 7.5, 6.6, 5.8, 4.9, 4.3, 4.0,
        # Pátý den - velmi klidný (horký den)
        2.1, 1.8, 1.5, 1.2, 1.0, 1.4, 2.8, 4.2, 5.4, 6.3, 6.8, 6.5,
        6.2, 5.9, 6.1, 6.6, 6.3, 5.7, 5.0, 4.2, 3.5, 2.9, 2.4, 2.2,
        # Šestý den - mírný vítr
        3.5, 3.1, 2.8, 2.4, 2.2, 2.6, 4.4, 6.1, 7.5, 8.8, 9.4, 9.1,
        8.7, 8.3, 8.6, 9.2, 8.8, 8.0, 7.2, 6.3, 5.5, 4.6, 4.0, 3.7,
        # Sedmý den - stabilní vítr
        3.9, 3.5, 3.2, 2.8, 2.6, 3.0, 4.8, 6.5, 7.9, 9.2, 9.8, 9.5,
        9.1, 8.7, 9.0, 9.6, 9.2, 8.4, 7.6, 6.7, 5.9, 5.0, 4.4, 4.1
    ]
})

# ----- ÚPRAVA FORMÁTU DAT -----

# hodinova_data['teplota'] = hodinova_data['teplota'].round(1)
# hodinova_data['vlhkost'] = hodinova_data['vlhkost'].round(0) 
# hodinova_data['rychlost_vetru'] = hodinova_data['rychlost_vetru'].round(1)

# ----- NASTAVENÍ ČASU JAKO INDEX -----
# Nastavení sloupce 'cas' jako index DataFrame pro snadnější práci s časovými řadami
hodinova_data.set_index('cas', inplace=True)
# set_index('cas') = použije sloupec 'cas' jako index místo číselných indexů 0,1,2...
# inplace=True = změní původní DataFrame, nevytváří kopii
# Index s časem umožňuje snadné filtrování podle období

# ----- ZOBRAZENÍ UKÁZKY DAT -----
print("Původní hodinová data (prvních 24 hodin):")
print(hodinova_data.head(24))
# .head(24) zobrazí prvních 24 řádků (celý první den)

print(f"\nCelkem záznamů: {len(hodinova_data)}")
# len() vrátí počet řádků v DataFrame (168 = 7 dní × 24 hodin)

# Shrnutí struktury dat:
# - Index: datetime (čas měření)
# - teplota: realistické hodinové teploty (°C)
# - vlhkost: relativní vlhkost vzduchu (%)
# - rychlost_vetru: rychlost větru (km/h)
# Všechna data odpovídají typickému letnímu týdnu s různými povětrnostními podmínkami

Původní hodinová data (prvních 24 hodin):
                     teplota  vlhkost  rychlost_vetru
cas                                                  
2023-06-01 00:00:00     15.2       82             3.2
2023-06-01 01:00:00     14.8       85             2.8
2023-06-01 02:00:00     14.5       87             2.5
2023-06-01 03:00:00     14.1       89             2.1
2023-06-01 04:00:00     14.0       91             1.9
2023-06-01 05:00:00     14.3       88             2.3
2023-06-01 06:00:00     15.8       75             4.1
2023-06-01 07:00:00     17.5       68             5.8
2023-06-01 08:00:00     19.8       58             7.2
2023-06-01 09:00:00     22.1       52             8.5
2023-06-01 10:00:00     24.3       47             9.1
2023-06-01 11:00:00     26.2       43             8.7
2023-06-01 12:00:00     27.8       41             8.3
2023-06-01 13:00:00     28.5       39             7.9
2023-06-01 14:00:00     28.9       38             8.2
2023-06-01 15:00:00     28.1       42   

### Resampling - snížení frekvence

In [3]:
# ----- ZÁKLADNÍ DENNÍ AGREGACE - PRŮMĚR -----
# resample('D') převede hodinová data na denní intervaly
# 'D' znamená Day (den), další možnosti: 'W' (týden), 'M' (měsíc), 'H' (hodina)
# .mean() vypočítá průměr všech hodnot v daném dni
denni_prumer = hodinova_data.resample('D').mean()
# Výsledek: místo 24 hodinových hodnot pro každý den dostaneme 1 průměrnou hodnotu

print("Denní průměry:")
print(denni_prumer)
# Zobrazí 7 řádků (jeden pro každý den) se třemi sloupci (teplota, vlhkost, rychlost_vetru)

# ----- RŮZNÉ AGREGACE PRO RŮZNÉ PROMĚNNÉ -----
# .agg() umožňuje použít různé agregační funkce pro různé sloupce
# Používáme slovník: klíč = název sloupce, hodnota = funkce nebo seznam funkcí
denni_souhrn = hodinova_data.resample('D').agg({
    'teplota': ['mean', 'min', 'max'],      # Pro teplotu: průměr, minimum a maximum
    'vlhkost': 'mean',                      # Pro vlhkost: pouze průměr
    'rychlost_vetru': ['mean', 'max']       # Pro vítr: průměr a maximum
})
# Výsledek je MultiIndex DataFrame s hierarchickými názvy sloupců

print("\nDenní souhrn s různými agregacemi:")
print(denni_souhrn)
# Ukáže například: teplota má 3 podsloupce (mean, min, max), vlhkost má 1 (mean)

# ----- VÝPOČET DENNÍ AMPLITUDY TEPLOT -----
# Amplituda = rozdíl mezi maximální a minimální teplotou během dne
# Nejprve resample().max() najde nejvyšší teplotu každého dne
# Pak resample().min() najde nejnižší teplotu každého dne
# Operátor - odečte minimum od maxima
denni_amplituda = hodinova_data['teplota'].resample('D').max() - \
                  hodinova_data['teplota'].resample('D').min()
# Výsledek je Series s jednou hodnotou (amplitudou) pro každý den

print("\nDenní teplotní amplituda:")
# Procházíme Series pomocí .items(), která vrací páry (datum, hodnota)
for datum, amplituda in denni_amplituda.items():
    # strftime formátuje datum do čitelného formátu DD.MM.YYYY
    print(f"{datum.strftime('%d.%m.%Y')}: {amplituda:.1f}°C")
    # {amplituda:.1f} zobrazí hodnotu na 1 desetinné místo

# Příklad výstupu:
# 01.06.2023: 14.9°C (rozdíl mezi 28.9°C a 14.0°C)
# 02.06.2023: 15.8°C (rozdíl mezi 30.5°C a 14.7°C)
# atd. pro všech 7 dní

Denní průměry:
              teplota    vlhkost  rychlost_vetru
cas                                             
2023-06-01  20.583333  65.833333        5.700000
2023-06-02  21.750000  62.041667        6.862500
2023-06-03  18.783333  83.500000        8.637500
2023-06-04  21.391667  67.791667        6.333333
2023-06-05  23.187500  59.666667        4.083333
2023-06-06  22.604167  64.458333        6.033333
2023-06-07  22.320833  66.333333        6.433333

Denní souhrn s různými agregacemi:
              teplota                vlhkost rychlost_vetru      
                 mean   min   max       mean           mean   max
cas                                                              
2023-06-01  20.583333  14.0  28.9  65.833333       5.700000   9.1
2023-06-02  21.750000  14.7  30.5  62.041667       6.862500  10.4
2023-06-03  18.783333  15.4  22.8  83.500000       8.637500  12.2
2023-06-04  21.391667  15.3  29.2  67.791667       6.333333   9.7
2023-06-05  23.187500  15.7  32.2  59.666667  

## Upsampling - zvýšení frekvence

### Základní metody

In [4]:

print("Denní průměry (výchozí data pro upsampling):")
print(denni_prumer)
print("\n" + "="*60 + "\n")

# ----- 1. FORWARD FILL (ffill) -----
# Převedeme denní data zpět na hodinové intervaly
# .resample('H') vytvoří hodinové časové razítko
# .ffill() vyplní prázdné hodnoty poslední známou hodnotou
hodinove_ffill = denni_prumer.resample('H').ffill()

print("1. FORWARD FILL - hodnota se 'protáhne' dopředu:")
print(hodinove_ffill.head(25))  # Zobrazí první den + 1 hodinu
print(f"\n→ Všech 24 hodin prvního dne má stejnou hodnotu: {hodinove_ffill.iloc[0,0]:.1f}°C")
print(f"→ Změna nastává až o půlnoci dalšího dne")

# ----- 2. BACKWARD FILL (bfill) -----
# .bfill() vyplní prázdné hodnoty následující známou hodnotou
hodinove_bfill = denni_prumer.resample('H').bfill()

print("\n\n2. BACKWARD FILL - hodnota se 'protáhne' dozadu:")
print(hodinove_bfill.head(25))
print(f"\n→ Prvních 23 hodin má hodnotu z NÁSLEDUJÍCÍHO dne: {hodinove_bfill.iloc[0,0]:.1f}°C")
print(f"→ Pouze poslední hodina dne (23:00) má hodnotu aktuálního dne")

# ----- POROVNÁNÍ METOD -----
print("\n\n" + "="*60)
print("POROVNÁNÍ FORWARD FILL vs BACKWARD FILL:")
print("-"*60)
print("Čas                  | Forward Fill | Backward Fill | Poznámka")
print("-"*60)

# Zobrazíme přechod mezi dny
for cas in pd.date_range('2023-06-01 22:00', '2023-06-02 02:00', freq='H'):
    ffill = hodinove_ffill.loc[cas, 'teplota']
    bfill = hodinove_bfill.loc[cas, 'teplota']
    
    if cas.hour == 0:
        poznamka = "← Změna u ffill"
    elif cas.hour == 23:
        poznamka = "← Poslední hodnota dne u bfill"
    else:
        poznamka = ""
    
    print(f"{cas} |    {ffill:.1f}°C    |    {bfill:.1f}°C     | {poznamka}")


Denní průměry (výchozí data pro upsampling):
              teplota    vlhkost  rychlost_vetru
cas                                             
2023-06-01  20.583333  65.833333        5.700000
2023-06-02  21.750000  62.041667        6.862500
2023-06-03  18.783333  83.500000        8.637500
2023-06-04  21.391667  67.791667        6.333333
2023-06-05  23.187500  59.666667        4.083333
2023-06-06  22.604167  64.458333        6.033333
2023-06-07  22.320833  66.333333        6.433333


1. FORWARD FILL - hodnota se 'protáhne' dopředu:
                       teplota    vlhkost  rychlost_vetru
cas                                                      
2023-06-01 00:00:00  20.583333  65.833333          5.7000
2023-06-01 01:00:00  20.583333  65.833333          5.7000
2023-06-01 02:00:00  20.583333  65.833333          5.7000
2023-06-01 03:00:00  20.583333  65.833333          5.7000
2023-06-01 04:00:00  20.583333  65.833333          5.7000
2023-06-01 05:00:00  20.583333  65.833333          5.7000

### Upsampling s redistribucí hodnot

### Expanding window

In [5]:
# ----- EXPANDING WINDOW - POSTUPNĚ SE ROZŠIŘUJÍCÍ OKNO -----
# Expanding window počítá statistiky od začátku časové řady až po aktuální bod
# Na rozdíl od rolling window (pevná velikost) se expanding window postupně zvětšuje

# ----- 1. ZÁKLADNÍ EXPANDING MEAN -----
# .expanding() vytvoří postupně se rozšiřující okno
# .mean() vypočítá průměr všech hodnot od začátku po aktuální bod
expanding_prumer = hodinova_data['teplota'].expanding().mean()

# První hodnota: průměr z 1 hodnoty = 15.2°C
# Druhá hodnota: průměr z prvních 2 hodnot = (15.2 + 14.8)/2 = 15.0°C
# Třetí hodnota: průměr z prvních 3 hodnot = (15.2 + 14.8 + 14.5)/3 = 14.83°C
# ... a tak dále až po poslední hodnotu, která je průměrem všech 168 hodnot

print("Expanding mean - ukázka prvních 5 hodnot:")
print(expanding_prumer.head())

# ----- 2. EXPANDING S VÍCE STATISTIKAMI -----
# .agg() umožňuje vypočítat více statistik najednou
# Každá statistika se počítá od začátku časové řady po aktuální bod
expanding_stats = hodinova_data['teplota'].expanding().agg(['mean', 'min', 'max'])

# mean: kumulativní průměr
# min: minimum ze všech dosavadních hodnot
# max: maximum ze všech dosavadních hodnot

print("\nExpanding statistiky - konec každého dne:")
# Zobrazíme hodnoty vždy o půlnoci (konec dne)
print(expanding_stats.iloc[23::24])  # Každý 24. řádek počínaje 23. (indexy 23, 47, 71...)

# ----- 3. EXPANDING PRO DETEKCI REKORDŮ -----
# Vytvoříme sloupec, který ukazuje, kdy bylo dosaženo nového maxima
expanding_max = hodinova_data['teplota'].expanding().max()
# Porovnáme aktuální teplotu s dosavadním maximem
nova_maxima = hodinova_data['teplota'] == expanding_max

# True znamená, že v dané hodině byla naměřena nejvyšší teplota od začátku měření
print("\nHodiny s novým teplotním maximem:")
print(hodinova_data[nova_maxima][['teplota']])

# ----- 4. EXPANDING S PARAMETREM min_periods -----
# min_periods určuje minimální počet hodnot potřebných pro výpočet
# Užitečné když chceme ignorovat první hodnoty (např. dokud nemáme alespoň 24 hodin dat)
expanding_prumer_24h = hodinova_data['teplota'].expanding(min_periods=24).mean()

# První 23 hodnot bude NaN (nedostatek dat)
# Od 24. hodnoty se začne počítat kumulativní průměr
print("\nExpanding mean s min_periods=24:")
print(expanding_prumer_24h.iloc[20:26])  # Ukázka přechodu z NaN na vypočítané hodnoty

# ----- 5. EXPANDING PRO KUMULATIVNÍ SOUČTY -----
# Představme si, že máme hodinové srážky a chceme kumulativní úhrn
# Pro demonstraci vytvoříme umělá data srážek
hodinova_data['srazky_mm'] = [0.5 if i in [42, 43, 67, 68, 69] else 0 for i in range(len(hodinova_data))]
# Srážky pouze v hodinách 42, 43 (2. den večer) a 67, 68, 69 (3. den večer)

kumulativni_srazky = hodinova_data['srazky_mm'].expanding().sum()
# .sum() sčítá všechny hodnoty od začátku

print("\nKumulativní srážky - vybrané hodiny:")
print(kumulativni_srazky.iloc[[40, 45, 70, 100]])
# Hodina 40: 0 mm (ještě nepršelo)
# Hodina 45: 1 mm (po prvním dešti)
# Hodina 70: 2.5 mm (po druhém dešti)
# Hodina 100: 2.5 mm (další déšť už nebyl)

Expanding mean - ukázka prvních 5 hodnot:
cas
2023-06-01 00:00:00    15.200000
2023-06-01 01:00:00    15.000000
2023-06-01 02:00:00    14.833333
2023-06-01 03:00:00    14.650000
2023-06-01 04:00:00    14.520000
Name: teplota, dtype: float64

Expanding statistiky - konec každého dne:
                          mean   min   max
cas                                       
2023-06-01 23:00:00  20.583333  14.0  28.9
2023-06-02 23:00:00  21.166667  14.0  30.5
2023-06-03 23:00:00  20.372222  14.0  30.5
2023-06-04 23:00:00  20.627083  14.0  30.5
2023-06-05 23:00:00  21.139167  14.0  32.2
2023-06-06 23:00:00  21.383333  14.0  32.2
2023-06-07 23:00:00  21.517262  14.0  32.2

Hodiny s novým teplotním maximem:
                     teplota
cas                         
2023-06-01 00:00:00     15.2
2023-06-01 06:00:00     15.8
2023-06-01 07:00:00     17.5
2023-06-01 08:00:00     19.8
2023-06-01 09:00:00     22.1
2023-06-01 10:00:00     24.3
2023-06-01 11:00:00     26.2
2023-06-01 12:00:00     27.8
2023

## Groupby s časovými daty

### Použití pd.Grouper()

In [ ]:
import pandas as pd
import numpy as np

# ----- VYTVOŘENÍ JEDNODUCHÝCH METEOROLOGICKÝCH DAT -----
# Data za 14 dní, měření každé 3 hodiny (8 měření denně)
# Celkem 112 záznamů (14 dní × 8 měření)

meteo_data = pd.DataFrame({
    'cas': pd.date_range('2024-06-01', periods=112, freq='3H'),
    'teplota': [
        # Týden 1 - teplejší
        15.2, 18.5, 22.3, 24.1, 23.8, 21.4, 18.7, 16.2,  # Den 1
        16.8, 19.6, 23.4, 25.2, 24.9, 22.5, 19.8, 17.3,  # Den 2
        17.5, 20.3, 24.1, 25.9, 25.6, 23.2, 20.5, 18.0,  # Den 3
        18.2, 21.0, 24.8, 26.6, 26.3, 23.9, 21.2, 18.7,  # Den 4
        17.9, 20.7, 24.5, 26.3, 26.0, 23.6, 20.9, 18.4,  # Den 5
        17.6, 20.4, 24.2, 26.0, 25.7, 23.3, 20.6, 18.1,  # Den 6
        17.3, 20.1, 23.9, 25.7, 25.4, 23.0, 20.3, 17.8,  # Den 7
        # Týden 2 - chladnější
        14.2, 16.5, 19.3, 21.1, 20.8, 18.4, 15.7, 13.2,  # Den 8
        13.8, 16.1, 18.9, 20.7, 20.4, 18.0, 15.3, 12.8,  # Den 9
        13.5, 15.8, 18.6, 20.4, 20.1, 17.7, 15.0, 12.5,  # Den 10
        13.2, 15.5, 18.3, 20.1, 19.8, 17.4, 14.7, 12.2,  # Den 11
        12.9, 15.2, 18.0, 19.8, 19.5, 17.1, 14.4, 11.9,  # Den 12
        12.6, 14.9, 17.7, 19.5, 19.2, 16.8, 14.1, 11.6,  # Den 13
        12.3, 14.6, 17.4, 19.2, 18.9, 16.5, 13.8, 11.3   # Den 14
    ],
    'vlhkost': np.random.randint(40, 90, size=112),
    'stanice': 'Praha'  # Jedna stanice pro jednoduchost
})

# ----- 1. PD.GROUPER S DENNÍ FREKVENCÍ -----
# pd.Grouper umožňuje flexibilní seskupování podle času
# key='cas' určuje sloupec s časovými razítky
# freq='D' znamená denní frekvenci (Day)
denni_prumery = meteo_data.groupby(pd.Grouper(key='cas', freq='D'))['teplota'].mean()

# Výsledek: 14 hodnot (jeden průměr pro každý den)
# Každý průměr je vypočítán z 8 měření daného dne

print("Denní průměrné teploty:")
print(denni_prumery.round(1))

# ----- 2. PD.GROUPER S TÝDENNÍ FREKVENCÍ -----
# freq='W' znamená týdenní frekvenci (Week)
# Týden končí v neděli
tydenni_statistiky = meteo_data.groupby(pd.Grouper(key='cas', freq='W'))['teplota'].agg(['mean', 'min', 'max'])

# Výsledek: 2 řádky (data pokrývají 2 týdny)
# První týden: 1.-7. června
# Druhý týden: 8.-14. června

print("\nTýdenní statistiky teplot:")
print(tydenni_statistiky.round(1))

# ----- 3. PD.GROUPER S 3-DENNÍMI INTERVALY -----
# freq='3D' znamená seskupení po 3 dnech
# Užitečné pro kratší předpovědi počasí
tridenni_prumery = meteo_data.groupby(pd.Grouper(key='cas', freq='3D'))['teplota'].mean()

# Výsledek: 5 období
# 1.-3. června, 4.-6. června, 7.-9. června, 10.-12. června, 13.-15. června

print("\nPrůměrné teploty po 3 dnech:")
for datum, teplota in tridenni_prumery.items():
    print(f"{datum.strftime('%d.%m.')}: {teplota:.1f}°C")

# ----- 4. PD.GROUPER S HODINOVÝMI INTERVALY (6H) -----
# freq='6H' seskupí data po 6 hodinách
# Vytvoří 4 období během dne: noc, ráno, odpoledne, večer
sesthodinove = meteo_data.groupby(pd.Grouper(key='cas', freq='6H'))['teplota'].mean()

# Ukážeme pouze první den
print("\nPrůměry po 6 hodinách (první den):")
print(sesthodinove.head(4).round(1))
# 00:00-06:00: průměr z měření v 0:00 a 3:00
# 06:00-12:00: průměr z měření v 6:00 a 9:00
# 12:00-18:00: průměr z měření v 12:00 a 15:00
# 18:00-24:00: průměr z měření v 18:00 a 21:00

# ----- 5. PD.GROUPER S PARAMETREM closed -----
# closed='right' znamená, že pravý okraj intervalu je uzavřený
# closed='left' znamená, že levý okraj intervalu je uzavřený (výchozí)
denni_right = meteo_data.groupby(pd.Grouper(key='cas', freq='D', closed='right'))['teplota'].count()

print("\nPočet měření v každém dni (closed='right'):")
print(denni_right.head(3))
# Ovlivňuje, do kterého dne patří půlnoční měření

# ----- 6. PD.GROUPER S VÍCE AGREGACEMI -----
# Můžeme použít více agregačních funkcí najednou
mesicni_souhrn = meteo_data.groupby(pd.Grouper(key='cas', freq='M'))['teplota'].agg([
    ('prumer', 'mean'),
    ('minimum', 'min'),
    ('maximum', 'max'),
    ('pocet_mereni', 'count')
])

# freq='M' znamená měsíční frekvenci (Month)
# Všechna naše data jsou z června, takže bude jen 1 řádek

print("\nMěsíční souhrn:")
print(mesicni_souhrn.round(1))